In [ ]:
import os
import yaml

from langchain.llms import OpenAI
from langchain import HuggingFaceHub

# LangChain Cheat Sheet

GitHub: https://github.com/hwchase17/langchain

Docs: https://python.langchain.com/en/latest/index.html

End-2-end example: https://github.com/langchain-ai/chat-langchain

LLM LocalRun: https://python.langchain.com/docs/guides/local_llms



**Content**
- **llms**
- **PromptTemplates**
- **Chains**
- **Agents and Tools**
- **Memory**
- **Document Loaders**
- **Indexes**

In [7]:
config = yaml.safe_load(open("..//config.yml"))
os.environ["OPENAI_API_KEY"] = config['openai_key']
os.environ["HUGGINGFACEHUB_API_TOKEN"] = config['huggingface_key']

## LLMs
### Generic Interface for LLMs

**LLM provibers:** https://python.langchain.com/docs/modules/model_io/llms/

In [23]:
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0.9)

In [24]:
llm("translate Ukrainian to English: Сьогодні я приїхав до Києва")

'\nToday I arrived in Kyiv. '

In [22]:
# https://huggingface.co/google/flan-t5-xl
# llm = HuggingFaceHub(repo_id="bert-base-uncased")
# llm("translate English to German: Today I arrived in Kiev")

## Prompt Templates

In [31]:
from langchain import PromptTemplate
template = "Question: {question} Let's think step by step. Answer: "

prompt = PromptTemplate(template=template, input_variables=["question"])

In [41]:
formated_str = prompt.format(question="Can Barack Obama have a conversation with George Washington?")

In [43]:
llm.predict(formated_str)

'No, it is not possible for Barack Obama to have a conversation with George Washington, as Washington passed away in 1799, long before Obama was born in 1961. They lived in different time periods and were never alive at the same time. '

## Chains

In [44]:
from langchain import LLMChain
llm_chain = LLMChain(prompt=prompt, llm=llm)
question = "Can Barack Obama have a conversation with George Washington?"
print(llm_chain.run(question))

Technically, no. Barack Obama and George Washington lived in different centuries and did not overlap in time. However, if we were to imagine a hypothetical scenario where they could have a conversation, it would be possible to do so through the use of technology and advanced historical research. Here is a potential step-by-step process:

1. Historical research: In order for this conversation to take place, thorough research would need to be conducted on both Barack Obama and George Washington. This would involve studying their backgrounds, beliefs, and major events in their lives.

2. Creation of a simulation: With the help of advanced technology, a simulation of both individuals could be created. This simulation would be based on historical data and would accurately depict their appearance, voice, and mannerisms.

3. Programming of conversation: Next, the conversation between the two individuals would need to be programmed. This would involve the use of artificial intelligence and nat

## Agents and Tools

In [50]:
# ! poetry add wikipedia
# ! poetry add numexpr
# ! pip install wikipedia

In [51]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI

In [52]:
llm = OpenAI(temperature=0)
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

In [53]:
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [54]:
agent.run("In what year was the film Departed with Leopnardo Dicaprio released? What is this year raised to the 0.43 power?")



> Entering new AgentExecutor chain...
 I need to find out the year the film was released and then use the calculator to calculate the power.
Action: Wikipedia
Action Input: Departed with Leonardo Dicaprio
Observation: Page: Leonardo DiCaprio filmography
Summary: Leonardo DiCaprio is an American actor who began his career performing as a child on television. He appeared on the shows The New Lassie (1989) and Santa Barbara (1990) and also had long running roles in the comedy-drama Parenthood (1990) and the sitcom Growing Pains (1991). DiCaprio played Tobias "Toby" Wolff opposite Robert De Niro in the biographical coming-of-age drama This Boy's Life in 1993. In the same year, he had a supporting role as a developmentally disabled boy Arnie Grape in What's Eating Gilbert Grape, which earned him nominations for the Academy Award for Best Supporting Actor and the Golden Globe Award for Best Supporting Actor – Motion Picture. In 1995, DiCaprio played the leading roles of an American author 

'The film Departed with Leonardo Dicaprio was released in 2006 and this year raised to the 0.43 power is 26.30281917656938.'

## Memory


In [55]:
from langchain import OpenAI, ConversationChain

llm = OpenAI(temperature=0)
conversation = ConversationChain(llm=llm, verbose=True)

conversation.predict(input="Hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


" Hi there! It's nice to meet you. How can I help you today?"

In [56]:
conversation.predict(input="Can we talk about AI?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: Can we talk about AI?
AI:

> Finished chain.


' Absolutely! What would you like to know about AI?'

In [57]:
conversation.predict(input="I'm interested in Reinforcement Learning.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: Can we talk about AI?
AI:  Absolutely! What would you like to know about AI?
Human: I'm interested in Reinforcement Learning.
AI:

> Finished chain.


' Sure! Reinforcement Learning is a type of machine learning algorithm that allows an AI agent to learn from its environment by taking actions and receiving rewards or punishments. It is used to solve complex problems that require trial and error. Would you like to know more about how it works?'

## Documant Loaders

Doc: https://python.langchain.com/en/latest/modules/indexes/document_loaders.html

In [58]:
from langchain.document_loaders import NotionDirectoryLoader

loader = NotionDirectoryLoader("Notion_DB")

docs = loader.load()

## Indexes

In [63]:
# ! poerty add sentence_transformers
# ! poetry add faiss-cpu

In [59]:
import requests

url = "https://raw.githubusercontent.com/hwchase17/langchain/master/docs/modules/state_of_the_union.txt"
res = requests.get(url)
with open("state_of_the_union.txt", "w") as f:
  f.write(res.text)

In [60]:
# Document Loader
from langchain.document_loaders import TextLoader
loader = TextLoader('./state_of_the_union.txt')
documents = loader.load()

In [61]:
# Text Splitter
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [65]:
# Embeddings
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

#text = "This is a test document."
#query_result = embeddings.embed_query(text)
#doc_result = embeddings.embed_documents([text])

.gitattributes: 100%|██████████| 1.18k/1.18k [00:00<00:00, 944kB/s]
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 292kB/s]
README.md: 100%|██████████| 10.6k/10.6k [00:00<00:00, 4.16MB/s]
config_sentence_transformers.json: 100%|██████████| 116/116 [00:00<00:00, 286kB/s]
data_config.json: 100%|██████████| 39.3k/39.3k [00:00<00:00, 374kB/s]
pytorch_model.bin:  89%|████████▊ | 388M/438M [00:52<00:05, 8.71MB/s] 

In [ ]:
# Vectorstore: https://python.langchain.com/en/latest/modules/indexes/vectorstores.html
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, embeddings)

query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)

In [ ]:
print(docs[0].page_content)

In [ ]:
db.save_local("faiss_index")
new_db = FAISS.load_local("faiss_index", embeddings)
docs = new_db.similarity_search(query)
print(docs[0].page_content)